In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape

def load_images(directory):
    images = []

    for filename in os.listdir(directory):
        if filename.endswith('.png'):
            file_path = os.path.join(directory, filename)
            try:
                img = Image.open(file_path).convert('RGB')
                img = img.resize((64, 64))  # Resize for uniformity
                img_array = np.array(img) / 255.0  # Normalize to [0, 1]
                images.append(img_array)
            except Exception as e:
                print(f"Error opening {filename}: {e}")

    return np.array(images)

def create_autoencoder(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D((2, 2), padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), padding='same'),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), padding='same'),

        UpSampling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        Conv2D(3, (3, 3), activation='sigmoid', padding='same')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

directory = 'faces/kawamura'
images = load_images(directory)

X_train, X_test = train_test_split(images, test_size=0.2, random_state=42)

autoencoder = create_autoencoder(input_shape=(64, 64, 3))
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test))

loss = autoencoder.evaluate(X_test, X_test)
print(f'Reconstruction Loss: {loss:.4f}')


Epoch 1/10


C:\Users\jadejaan\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 375ms/step - loss: 0.6911 - val_loss: 0.6240
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 177ms/step - loss: 0.6105 - val_loss: 0.6138
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - loss: 0.6129 - val_loss: 0.5824
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - loss: 0.5805 - val_loss: 0.5466
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - loss: 0.5464 - val_loss: 0.5210
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - loss: 0.5318 - val_loss: 0.4877
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 255ms/step - loss: 0.4880 - val_loss: 0.4643
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - loss: 0.4702 - val_loss: 0.4493
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 201ms/step - loss: 0.4539 - val_loss: 0.4439
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 190ms/step - loss: 0.4450 - val_loss: 0.4383
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.4383
Reconstruction Loss: 0.4383
